# Homework 4: Transfer learning with the inception network

This is Homework #4. My name is Yiqiao Yin and my UNI is yy2502. I approve this message.

## DUE Friday 17 November at 4pm (both sections)
Submit your homework by publishing a notebook that cleanly displays your code, results, and plots to pdf or html

## Description

In this homework, we will fine-tune the existing [inception network](https://arxiv.org/pdf/1512.00567.pdf) to classify some new categories of images. Building and fitting the network from scratch is expensive and far beyond the scope of this assignment, so you will load the tensorflow graph and variables which have been pre-trained on the imagenet dataset. You are responsible for completing the skeleton code in this notebook in order to use tensorboard to investigate the graph architecture, gain insight into the behavior of the pre-trained network on a new image classification task, fine-tune the network for this new task (retraining the last layer), and finally evaluate the performance of the fine-tuned model.

## Downloading the network and data

In order to avoid clutter and extraneous technical details, we have created a supplementary module  `transfer_learning.py` that does most of the heavy lifting. We have only exposed the high-level which you are responsible for completing and understanding in full in this notebook.  While you are encouraged to read `transfer_learning.py`, you should not need to modify it in any way.  To import this module into colab, you must first execute the cell below, and then second you must upload the transfer_learning.py file using the upload button that will appear.

In [84]:
# Import transfer_learning module. 
from google.colab import files
src = list(files.upload().values())[0]
open('transfer_learning.py','wb').write(src)
import transfer_learning

Saving transfer_learning.py to transfer_learning (9).py


In [0]:
# Import the rest
import tensorflow as tf
import numpy as np
import os
import os.path

We will be using tensorboard to visualize the network, inputs, and other summaries of the training process. After running the following cell, follow the instructions for using tensorboard, evailable in Andrew's excellent tensorflow tutorial.

In [0]:
# Ensure target log dir exists
INCEPTION_LOG_DIR = './tmp/inception_v3_log'
if not os.path.exists(INCEPTION_LOG_DIR):
    os.makedirs(INCEPTION_LOG_DIR)

Your goal in the assignment is to use a small collection of photos (500 to be exact) to construct a flower-image classifier capable of discerning differences between daisies, roses, dandelions, sunflowers, and tulips. This presents a obstacle to the use of state-of-the-art image classification models due to the heavily underdetermined nature of the problem (# features = #pixels * #channels >> #images). Moreover, training a large convolutional neural network from the ground up is can be computationally prohibitive. Alternatively, strongly biased linear classifiers like regularized logistic regression avoid these pitfalls. However, the performance of these models will be inadequate, because most important image features are non-linear transformations of the raw input and it is not obvious what these non-linear transformations should be.

To circumvent these challenges, we will leverage what a pre-trained convolutional neural network has already learned about important image features from the [imagenet dataset](http://www.image-net.org/) by fine tuning it to our specific problem. To begin, download the the flower dataset [here](http://download.tensorflow.org/example_images/flower_photos.tgz) and extract the contents into a sub-folder of your working directory named ```data```. This dataset is from a [Google Code Lab](https://codelabs.developers.google.com/codelabs/tensorflow-for-poets/#0), though note that we ask you to do considerably more than in that lab, so it is not helpful (nor are the commands covered therein valid answers to this homework).

The following cell will partition the images into a training and testing set, then it will load lists of image paths for each.

In [0]:
# download and extract flower data
transfer_learning._maybe_download_and_extract('http://download.tensorflow.org/example_images/flower_photos.tgz')

In [88]:
# For performance reasons, we only use 100 images per class.
training_images, testing_images, label_maps = transfer_learning.create_image_lists(
    './model/flower_photos', testing_percentage=10, max_number_images=100)

INFO:tensorflow:Looking for images in 'sunflowers'
INFO:tensorflow:Looking for images in 'dandelion'
INFO:tensorflow:Looking for images in 'daisy'
INFO:tensorflow:Looking for images in 'roses'
INFO:tensorflow:Looking for images in 'tulips'


Next, we will load the pre-trained inception model and look at its architecture in tensorboard. 

#### 1). Complete the following code block by opening a filewriter and pass it the graph object we have created.

In [0]:
# Create the inception model.
# Warning! The first time you run this, it will try to download the inception model
# from the internet. This is somewhat large (80MB), and can sometimes fail. If
# it does, simply try again.

graph, bottleneck, resized_input, softmax = transfer_learning.create_model()

In [0]:
# Use a summary writer to write the loaded graph and display it in tensorboard.
# Then answer the questions below
with graph.as_default():
    jpeg_data, decoded_image = transfer_learning.make_jpeg_decoding()
    #-- Students Complete
    sess = tf.Session()
    writer = tf.summary.FileWriter(INCEPTION_LOG_DIR, sess.graph)
    #--

In [0]:
# Run Tensorflow in the background - note that we specify the log 
# directory we want to look at
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(INCEPTION_LOG_DIR)
)

In [81]:
# Download and unzip ngrok - you will only need to do this once per session
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

--2018-11-17 16:42:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.173.6.94, 34.196.237.103, 52.204.188.97, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.173.6.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]   5.11M  --.-KB/s    in 0.1s    

2018-11-17 16:42:40 (42.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [0]:
# Launch the ngrok background process
get_ipython().system_raw('./ngrok http 6006 &')

In [93]:
# Get the public URL and be sorted!
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://e1cd9ec8.ngrok.io


#### 2.) In tensorboard, expand the 'softmax' and then the 'logits' block in order to answer to following questions:
* What are the dimensions of the inputs/outputs to/from the softmax block?
* What are the shape and dimension of the input images?
* When considered in isolation, what model is the 'softmax' block implementing? Explain.

In [0]:
# Download and unzip ngrok - you will only need to do this once per session
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip


In [0]:
# Launch the ngrok background process
get_ipython().system_raw('./ngrok http 6006 &')


In [77]:
# Get the public URL and be sorted!
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


http://e1cd9ec8.ngrok.io


Support your answers by including screenshots from tensorboard. (to insert images, simply copy and paste into the notebook)



Write your answers here.

- Dimension of inputs: 1x2048;
- Dimension of output: 1x1008.
- The shape and dimension of the inpute images are 299x299x3.
- We can use logistic regression.

## Classifying out of the box

Before retraining or modifying the network, we would like to better understand how it behaves on our dataset out of the box. Therefore, we will use it classify a couple of our flower images. First, we must create summary operations to write quanities of interest to tensorboard so that we may investigate. 

#### 3.) Complete the block below by implementing the following summaries:
* `output_summary`: a summary of output probabilities (histogram of the output of the softmax block)
* `input_summary`: a summary of image input to the network (image of resized and decoded jpeg data)
* `bottleneck_summary`: a summary of bottleneck activations (histogram of the input to the softmax block)

In [0]:
with graph.as_default():
    # Define summaries for tensorboard
    #-- Students complete
    # histogram of softmax output
    
    # display image of input
    
    # second to last layer output
    with tf.Session() as sess:
        output_summary = tf.summary.histogram("output_summary", softmax)
        input_summary = tf.summary.image("input_summary", resized_input)
        bottleneck_summary = tf.summary.histogram("bottleneck_summary", bottleneck)
    #-- 
    summary_op = tf.summary.merge_all()

Next, we will need a function that streamlines the process of performing inference given an image. This function will take as input the tensorflow session, a path to the image, and the ```summary_op``` we have created above. It will then open a ```t.summary.FileWriter``` and run the network forwards to obtain the formatted image, softmax output, and summaries. 

#### 4). Complete the code block below as instructed in each of the comments.

In [0]:
def classify_image(session, image_path, summary_op):
    """This functions reads a single image from disk and classifies
    the image using the pre-trained network.
    
    Parameters
    ----------
    session: the tensorflow session to use to run the operation
    image_path: a string corresponding to the path of the image to load
    summary_op: the summary operation.
    
    Returns
    -------
    label: an integer representing the label of the classified example.
    softmax_output: the network's output multinomial probabilities
    """
    
    # Open single image file and extract data
    with open(image_path, 'rb') as f:
        image_data = f.read()
    
    #-- Students Complete
    # Create Summary writer object
    writer = tf.summary.FileWriter(INCEPTION_LOG_DIR)
    
        
    # Run pre-defined pipeline to decode & resize raw image data into decoded_image 
    resized_input_values = session.run(decoded_image,
                                  {jpeg_data: image_data})
    
        
    # Run input through network and obtain output from last layer, defined by create_model() call above.
    # Note: be sure to run your summary operation as well.
    bottleneck_values = session.run(bottleneck,
                               {resized_input: resized_input_values})
    softmax_output = session.run(softmax,
                               {bottleneck: bottleneck_values})
    summary_out = session.run(summary_op, {resized_input: resized_input_values, bottleneck: bottleneck_values, softmax:softmax_output})
    
    
    # Log summary & terminate writer in the usual way.
    writer.add_summary(summary_out)
    writer.close()
   
  
    #--
    
    # Return label
    return(np.argmax(softmax_output),softmax_output)

We are now ready to classify some images. Run the code in this cell in order to obtain the output label and navigate to tensorboard to look at the histogram and image summaries that have been logged. **Note:** You may need to refresh Tensorboard in order for this to display.

In [0]:
image_path = os.path.join(os.getcwd(), 'model', 'flower_photos', 'daisy', '100080576_f52e8ee070_n.jpg')
with graph.as_default():
    with tf.Session() as session:
        # We classify the image and print the label
        imagenet_label,_ = classify_image(session, image_path, summary_op)
print(imagenet_label)

#### 5.) Confirm that your code works by taking a screenshot of the input image in tensorboard.

Include your screenshot here.

If you've completed the prevous steps correctly, you should see that the pre-trained network assigns the label 357 to our image. Cross referencing this with a [source](https://gist.github.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57) that explains what each of the imagenet labels are, we see that our classifier is right on target. The image was indeed a daisy, but this is solely a consequence of the fact that the class 'daisy' is included in the imagenet dataset. We should not expect such favorable outcome when this is not the case. We will test this assumtion by classifying a different type of flower image.

In [0]:
image_path = os.path.join(os.getcwd(), 'model', 'flower_photos', 'roses', '19440805164_920b28da61_n.jpg')
with graph.as_default():
    with tf.Session() as session:
        imagenet_label,softmax_output = classify_image(session, image_path, summary_op)
print(imagenet_label)

In [0]:
max(softmax_output[0])

#### 6.) Please explain what you have found by answering the following questions:

* What does the network think this is an image of? Is this reasonable?
* How confident is it in assigning this label (what is the probability it assigns)?
* Why might the network behave this way?
* What warning might you give about deep-learning/machine-learning in general based on these observations?

Support your answers by referencing screenshots from tensorboard (of the input picture, softmax output, etc.).

Answer these questions here.

- Dimension of inputs: 1x2048;
- Dimension of output: 1x1008.
- The shape and dimension of the inpute images are 299x299x3.
- We can use logistic regression.

## Adapting The Network

Clearly, this model will not suffice for our application out of the box. However, it is not unreasonable to think that we may be able to adapt what this network has "learned" about processing images for our purposes. Remember that the last layer of the network implements a well known classifier (see your answer to question 2). Under this interpretation, the layers of the network up until the last are responsible for learning a non-linear mapping between raw inputs (pixel values) and useful derived features. Consequently, we may remove the last layer, fix the weights of the remaining layers, use what remains as a black-box function transforming images into derived feature vectors, and finally fit a new classifier on the derived feature vectors. 

### Pre-Computing the bottlenecks

To train the new classifier efficiently, we will want to pre-compute the derived feature vectors. We have been calling these the 'bottleneck', because they would limit the speed of training if we had to be re-computed them iteration. Additionally, its small size relative to the dimension of our raw input is essential in ensures that it provides a compact summary of the images for our classifier.

#### 7). Which block in the tensorboard graph is the bottleneck?

Write you answer here.

-The bottleneck is the pool_3 block.

We have written code that will cache the derived features or 'bottleneck activation' for each images. In order to use this, we require that you complete a simple function that returns these activations.

#### 8). Complete the code block below and compute the bottlenecks.

In [0]:
def compute_bottleneck(session, image_data):
    """Computes the bottleneck for a given image
    
    Parameters
    ----------
    session: the tensorflow session to use for the computation.
    jpeg_data_tensor: the tensor to feed the jpeg data into.
    bottleneck_tensor: the tensor representing.
    image_data: a byte sequence representing the encoded jpeg data.
    
    Returns
    -------
    A numpy array containing the bottleneck information for the image.
    """
    #-- To be completed by the student
    num = session.run(decoded_image, feed_dict={jpeg_data: image_data})
    bottleneck_output = session.run((bottleneck), feed_dict={resized_input: num})
    return(bottleneck_output) 
    
    #--

In [0]:
# This cell generates all the bottlenecks. Warning: it may take a while
# The results are cached so you only need to do it once -- if you change
# your compute_bottleneck function, you will need to delete the existing
# files to force the notebook to regenerate the bottlenecks (they are
# found in ./data/bottlenecks)

with graph.as_default():
    with tf.Session() as session:
        transfer_learning.cache_bottlenecks(compute_bottleneck, session, training_images)

Now that these have been saved, we can load them in a format that is ammenable to use by tensorflow.

In [0]:
# This loads the training data as a matrix of training examples
# and a vector of labels
training_data_set = transfer_learning.create_training_dataset(training_images)

### Training the classifier

We are finally ready to train the new classifier. This is equivalent to replacing & retraining the final layer of our original network with a different number of output classes, so we will still be using tensorflow.

#### 9). Complete the following function as instructed in the comments by:
* Creating a new dense layer of weights, biases, and logits.
* Computing/summarizing the cross-entropy loss.
* Creating an optimizer/training step.

In [0]:
def make_final_layers(bottleneck_tensor, num_classes):
    """Create the operations for the last layer of the network to be retrained.
    
    This function should implement a logistic regression layer from the bottleneck
    to the labels, trained using gradient descent. We have created the inputs
    bottleneck_input and label_input for you. You are responsible for implementing
    the logistic regression layer itself (as you have seen in lecture),
    the predicted_output (predicted probability for each class), the cross entropy
    loss, and the optimization and subsequent training step operation (an
    operation that executes one step of gradient descent).  Note: be careful to
    use the numerically stable implementation of cross entropy loss as discussed
    in lecture.
    
    You should also record a summary for the loss you compute.
    
    Parameters
    ----------
    bottleneck_tensor: the bottleneck tensor in the original network
    num_classes: the number of output classes
    
    Returns
    -------
    bottleneck_input: the input placeholder for the bottleneck values
    label_input: the input placeholder for the label values
    logits: the tensor representing the unnormalized log probabilities
        for each class.
    train_step: an operation representing one gradient descent step.
    """
    bottleneck_tensor_size = int(bottleneck.shape[1])
    
    with tf.variable_scope('input'):
        # This is the input for the bottleneck. It is created
        # as a placeholder with default. During training, we will
        # be passing in the bottlenecks, but during evaluation,
        # the value will be propagated from the bottleneck computed
        # from the image using the full network.
        bottleneck_input = tf.placeholder_with_default(
            bottleneck_tensor,
            [None, bottleneck_tensor_size],
            'bottleneck_input')
        
        # This is the input for the label (integer, 1 to number of classes)
        label_input = tf.placeholder(tf.int64, [None], name='label_input')
    
    
    # -- Start student must write
    # Define weights, biases, and logit transforms
    logits = tf.layers.dense(bottleneck_input, num_classes)
    
    # Compute the cross entropy loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=label_input, logits=logits)
    
    # Create a summary for the loss
    loss_summary = tf.summary.scalar('cross_entropy', loss)
    
    # Create a Gradient Descent Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.1)
    
    # Obtain a function which performs a single training step
    train_step = optimizer.minimize(loss)
    
    # -- End student must write
    
    return bottleneck_input, label_input, logits, train_step, loss_summary

We will also write a function to evaluate the model's classification accuracy.

#### 10). Complete the following function as instructed in the comments.

In [0]:
def compute_accuracy(labels, logits):
    """Compute the accuracy for the predicted output.
    
    Parameters
    ----------
    labels: The input labels (in a one-hot encoded fashion).
    predicted_output: The predicted class probability for each output.
    
    Returns
    -------
    A tensor representing the accuracy.
    """
    #-- To be written by student
    # Collapse logit output to scalar labels
    
    # Compute accuracy
    with tf.name_scope("accuracy"):
        pred = tf.argmax(logits, 1, name="pred_class")
        accuracy = tf.reduce_mean(tf.cast(tf.equal(labels, pred), dtype=tf.float32))
    # Create summary of accuracy
    with tf.variable_scope("summary"):
        accuracy_summary = tf.summary.scalar("accuracy", accuracy)
    #--
    
    return accuracy, accuracy_summary

These functions have been completed for you. Make sure you understand what they are doing as it will help you figure out what to write in the functions above.

In [0]:
# We create the necessary operations to fine tune the model.

with graph.as_default():
    bottleneck_input, label_input, logits, train_step, loss_summary = make_final_layers(bottleneck, len(label_maps))
    accuracy, accuracy_summary = compute_accuracy(label_input, logits)
    summary_op = tf.summary.merge([loss_summary, accuracy_summary])

In [0]:
def execute_train_step(session: tf.Session, summary_writer: tf.summary.FileWriter, current_step: int):
    """This function runs a single training step.
    
    You may wish to print some progress information as you go along.
    
    Parameters
    ----------
    session: the tensorflow session to use to run the training step.
    summary_writer: the summary file writer to write your summaries to.
    current_step: the current step count (starting from zero)
    """
    _, ac, summary = session.run((train_step, accuracy, summary_op),
                       feed_dict={bottleneck_input: training_data_set['bottlenecks'],
                                  label_input: training_data_set['labels']
                                 })
    
    summary_writer.add_summary(summary, current_step)
    
    if current_step % 10 == 0:
        print('Accuracy at step {0} is {1}'.format(current_step, ac))
        

In [0]:
def evaluate_images(session: tf.Session, images_jpeg_data: [bytes], labels: [int]):
    """This function will evaluate the accuracy of our model on the specified data.
    
    Parameters
    ----------
    session: the tensorflow session to use to run the evaluation step.
    images_jpeg_data: a list of strings, with each element in the list corresponding
        to the jpeg-encoded data for a given image
    labels: a list of integers, with each element in the list corresponding to the label
        of a given image.
    
    Returns
    -------
    This function should return the accuracy as a floating point number between
    0 and 1 (proportion of correctly classified instances).
    """
    correct = []
    
    for label, jpeg in zip(labels, images_jpeg_data):
        image_data = session.run(decoded_image, feed_dict={jpeg_data: jpeg})
        ac = session.run(accuracy, feed_dict={resized_input: image_data, label_input: [label]})
        correct.append(ac)
    
    return np.mean(correct)

Now, we put these functions together to train the final layer. Run the cell and confirm that your functions are working correctly (the accuracy on the training set should be increasing).

#### 11). Navigate to tensorboard and use what you find to answer the follow questions:
* What is the accuracy on the test set? How does this compare to random guessing?
* How do the training loss and accuracy progress with the number training iterations?
* What is the final loss/training error? What might this make you suspicious of? Can you provide a mathematical explanation for why this is happening?
* What might you change in order to increase the classification performance of this model?

As always, support your with screenshots from tensorboard and references to specific quantitative results.

In [0]:
# Download and unzip ngrok - you will only need to do this once per session
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip


In [0]:
# Launch the ngrok background process
get_ipython().system_raw('./ngrok http 6006 &')


In [0]:
# Get the public URL and be sorted!
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
